ipython kernel install --name "cac2020" --user


In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [25]:
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dense, Dropout, Flatten
from tensorflow_core.python.keras.callbacks import ModelCheckpoint

In [26]:
# from keras_tdqm import TQDMNotebookCallback
import json

In [27]:
train_image_path = "C:\\Users\\neelj\\Documents\\Python\\fruits-360" \
                   "\\modified-sets\\Training-modified"
test_image_path = "C:\\Users\\neelj\\Documents\\Python\\fruits-360" \
                  "\\modified-sets\\Test-modified"

batch_size = 128
num_classes = 56
num_epochs = 20
img_rows, img_cols = 100, 100
optimizer = Adam()

In [28]:
train_data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
)
validation_data_gen = ImageDataGenerator(rescale=1./255,
                                         validation_split=0.2,)

train_data_gen = train_data_gen.flow_from_directory(
    train_image_path,
    # save_to_dir="C:\\Users\\neelj\\Documents\\Python\\fruits-360" \
    # "\\modified-sets\\training-processed",
    # save_format="jpeg",
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
)

validation_data_gen = validation_data_gen.flow_from_directory(
    train_image_path,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
)

Found 23048 images belonging to 56 classes.
Found 5743 images belonging to 56 classes.


In [30]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(2, 2),
                 activation="relu", input_shape=(img_cols, img_rows, 3),
                padding='same'))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(64, kernel_size=(2, 2), activation="relu", padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(128, kernel_size=(2, 2), activation="relu", padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(150, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation="softmax"))

model.compile(loss="categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"],)

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 100, 100, 16)      208       
_________________________________________________________________
activation_2 (Activation)    (None, 100, 100, 16)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 50, 50, 64)        4160      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 25, 25, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 25, 25, 128)       32896     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 12, 12, 128)      

In [31]:
"""
CNN_model = model.fit(
    train_data_gen,
    epochs=num_epochs,
    shuffle=True,
    verbose=2,
    validation_data=validation_data_gen,
)
"""

CNN_model = model.fit_generator(
    train_data_gen,
    steps_per_epoch=train_data_gen.samples // batch_size,
    validation_data=validation_data_gen,
    validation_steps=validation_data_gen.samples // batch_size,
    epochs=num_epochs,
    verbose=1,
)

Epoch 1/20
180/180 [==============================] - 193s 1s/step - loss: 1.4291 - accuracy: 0.6171 - val_loss: 0.0637 - val_accuracy: 0.9592
Epoch 2/20
180/180 [==============================] - 184s 1s/step - loss: 0.1750 - accuracy: 0.9459 - val_loss: 0.1645 - val_accuracy: 0.9491
Epoch 3/20
180/180 [==============================] - 185s 1s/step - loss: 0.0895 - accuracy: 0.9703 - val_loss: 0.3129 - val_accuracy: 0.9517
Epoch 4/20
180/180 [==============================] - 203s 1s/step - loss: 0.0528 - accuracy: 0.9826 - val_loss: 0.4433 - val_accuracy: 0.9671
Epoch 5/20
180/180 [==============================] - 217s 1s/step - loss: 0.0435 - accuracy: 0.9860 - val_loss: 0.2168 - val_accuracy: 0.9719
Epoch 6/20
180/180 [==============================] - 251s 1s/step - loss: 0.0366 - accuracy: 0.9882 - val_loss: 0.0956 - val_accuracy: 0.9697
Epoch 7/20
180/180 [==============================] - 254s 1s/step - loss: 0.0309 - accuracy: 0.9888 - val_loss: 0.4617 - val_accuracy: 0.9605

In [32]:
test_data_gen = ImageDataGenerator(rescale=1./255)

test_data_gen = test_data_gen.flow_from_directory(
    train_image_path,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode="categorical",
)

model.evaluate_generator(test_data_gen,
                        verbose=1,)

Found 28791 images belonging to 56 classes.
225/225 [==============================] - 60s 265ms/step


[0.00014676537830382586, 0.9952762722969055]

In [33]:
idx_to_cls = {v: k for k, v in train_data_gen.class_indices.items()}
print(idx_to_cls)

{0: 'Apple Golden', 1: 'Apple Granny Smith', 2: 'Apple Red Delicious', 3: 'Apple Red Yellow', 4: 'Apricot', 5: 'Avocado', 6: 'Banana', 7: 'Beetroot', 8: 'Blueberry', 9: 'Cantaloupe', 10: 'Cauliflower', 11: 'Cherry', 12: 'Chestnut', 13: 'Coconut', 14: 'Corn', 15: 'Dates', 16: 'Eggplant', 17: 'Fig', 18: 'Grape Pink', 19: 'Grape White', 20: 'Guava', 21: 'Hazelnut', 22: 'Kiwi', 23: 'Kumquats', 24: 'Lemon', 25: 'Limes', 26: 'Lychee', 27: 'Mandarine', 28: 'Mango', 29: 'Nectarine', 30: 'Nut Forest', 31: 'Nut Pecan', 32: 'Onion Red', 33: 'Onion White', 34: 'Orange', 35: 'Papaya', 36: 'Passion Fruit', 37: 'Peach', 38: 'Pear', 39: 'Pear Red', 40: 'Pepper Green', 41: 'Pepper Orange', 42: 'Pepper Red', 43: 'Pepper Yellow', 44: 'Pineapple', 45: 'Plum', 46: 'Pomegranate', 47: 'Potato Red', 48: 'Potato White', 49: 'Raspberry', 50: 'Redcurrant', 51: 'Strawberry', 52: 'Tomato', 53: 'Tomato Cherry Red', 54: 'Walnut', 55: 'Watermelon'}


In [34]:
with open("class-indicies.json", "w") as f:
    json.dump(idx_to_cls, f, indent=4)

In [40]:

predict_generator = ImageDataGenerator(rescale=1/255.)

predict_generator = predict_generator.flow_from_directory("C:\\Users\\neelj\\PycharmProjects\\cac2020\\classifier",
                              classes=["test-images"],
                              class_mode=None,
                              shuffle=False,
                              target_size=(img_rows, img_cols),
                              )

predictions = model.predict_generator(predict_generator)

print(predictions)
preds_cls_idx = predictions.argmax(axis=1)
print(preds_cls_idx)

Found 1 images belonging to 1 classes.
[[1.06971645e-11 1.36226442e-18 2.04950001e-10 1.77623619e-07
  2.26525753e-20 1.26678765e-10 9.98051763e-01 6.10852896e-11
  1.22089852e-13 1.02030847e-03 8.98550206e-04 2.48573487e-13
  2.37947704e-24 7.29672721e-15 7.99509010e-08 1.17874244e-07
  2.01718775e-09 1.25215638e-09 1.50966814e-25 8.93102083e-07
  1.48295503e-18 2.77710897e-05 1.79381684e-23 2.33018355e-10
  6.56911439e-18 1.24833963e-11 3.57217917e-14 6.65476936e-18
  3.01481960e-18 7.81025523e-16 2.48731287e-18 2.67684919e-09
  3.03733174e-07 2.76384465e-13 2.00938688e-22 5.41490604e-21
  1.80977449e-14 9.28639000e-21 1.07702918e-15 1.00583886e-12
  4.00197444e-20 4.29368346e-11 1.41313575e-11 2.87936143e-19
  7.84989563e-15 7.70754851e-17 4.12902867e-19 5.09334974e-09
  6.07906070e-16 8.14255347e-12 9.80767696e-20 7.01418047e-12
  7.21000724e-24 3.93364509e-16 2.95850012e-21 1.27808387e-12]]
[6]


In [41]:
model_json = model.to_json()
with open("model.json", "w") as f:
    f.write(model_json)
model.save_weights("model.h5")

ALTERNATIVE MODEL Design that may work idk